# Default on credit card prediction (SVM) 

Let's introduce at first the dataset that would be used in this mini-project. (You'll find the source at the Read-me of the repo). 

So let's visualize this dataset and define attributes as we go on. 

In [2]:
import numpy as np
import pandas as pd
from sklearn import svm, datasets

In [3]:
input_file = "C:/Users/YsfEss/Desktop/UCI_Credit_Card.csv"
df = pd.read_csv(input_file, header = 0)
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


There is 25 attributes for each client, but we will be interested in particular in 12 attributes (11 predictors & 1 output) for building our model. The features used for the rest of this tutorial are: 


**1- ID: ID of each client** (Won't be used as feature obviously, I left it for visualization purposes only)

**2- LIMIT_BAL: Amount of given credit in NT dollars**

**3- SEX: Gender (1=male, 2=female)**

**4- EDUCATION: (1=graduate school, 2=university, 3=high school, 4=others, 5=unknown, 6=unknown)**

**5- MARRIAGE: Marital status (1=married, 2=single, 3=others)**

**6- AGE: Age in years**

for i [0,6]

**(7-12)- PAY_i: Repayment status in i+9 month, 2005 (-1=pay duly, 1=payment delay for one month, 2=payment delay for two months, ... 8=payment delay for eight months, 9=payment delay for nine months and above)**

**default.payment.next.month: Default payment (1=yes, 0=no)**


In [15]:
features=df.columns
features=list(features)
features=features[0:12]+[features[len(features)-1]]
df1=df[features]
df1

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,1
1,2,120000.0,2,2,2,26,-1,2,0,0,0,2,1
2,3,90000.0,2,2,2,34,0,0,0,0,0,0,0
3,4,50000.0,2,2,1,37,0,0,0,0,0,0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,0,0,0
5,6,50000.0,1,1,2,37,0,0,0,0,0,0,0
6,7,500000.0,1,1,2,29,0,0,0,0,0,0,0
7,8,100000.0,2,2,2,23,0,-1,-1,0,0,-1,0
8,9,140000.0,2,3,1,28,0,0,2,0,0,0,0
9,10,20000.0,1,3,2,35,-2,-2,-2,-2,-1,-1,0
